# View GRAB mismatch regular and closed loop data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy.stats import pearsonr, spearmanr
from analysis_functions import *
from model_functions import *
import matplotlib.patches as patches
import seaborn as sns

In [ ]:
RunTresholdDict = {'B3M1': 145, 'B3M2': 295, 'B3M3': 325, 'B2M4': 110, 'B2M5': 180}

In [ ]:
GRAB_Reg_1 = pd.read_csv('/Path/To/Session/csvfile.csv')#, dtype=dtype_dict)
GRAB_Reg_2 = pd.read_csv('/Path/To/Session/csvfile.csv'')#, dtype=dtype_dict)

In [ ]:
#Make sure the index provides inforamtion of seconds since start
GRAB_Reg_1.set_index('Seconds', inplace=True)
GRAB_Reg_2.set_index('Seconds', inplace=True)



In [ ]:
#Make sure theres a column for session number:
if 'session' not in GRAB_Reg_1.columns:
    GRAB_Reg_1['Session'] = 'day1'
if 'session' not in GRAB_Reg_2.columns:
    GRAB_Reg_2['Session'] = 'day2'

In [ ]:
#Have a look that the data is as expected
GRAB_Reg_1

In [ ]:
# Chaning event name to halt and making it so that its True when there is a halt
#GRAB_Reg_1.loc[:, 'event'] = GRAB_Reg_1['event'].replace({False: True, True: False})
GRAB_Reg_1.rename(columns = {'event': 'halt'}, inplace = True)
GRAB_Reg_2.rename(columns = {'event': 'halt'}, inplace = True)

In [ ]:
#Check that the same mice are in the datsets
print(GRAB_Reg_1.mouseID.unique())
print(GRAB_Reg_2.mouseID.unique())
names = GRAB_Reg_2.mouseID.unique()

# View session data
- Loop through the mouse names present in one of the dataframes (ideally they should be the same)
- save a variable where each session is saved for the current mouse
- use the view_session_mouse() funciton from analysis_functions.py to plot the Delta F/F 470 fluorescence and movement in X direction with halts in grey, and session blocks marked in colors.
- Can also add other column names to the plotlist, like eye movements and other fluorescent traces
- This is mostly to get an impression of the overall data trends.

Move the function into this file to test it.




In [ ]:
for mouse in GRAB_Reg_1.mouseID.unique():
    mousedata = {'session 1':GRAB_Reg_1.loc[GRAB_Reg_1.mouseID == mouse], 'session 2':GRAB_Reg_2.loc[GRAB_Reg_2.mouseID == mouse]}
    fig, ax = view_session_mouse(mousedata, mouse)
    fig.savefig(f'Figures/GRAB{mouse}_view_alignment.png', format = 'png', dpi = 300)

## Filter with respect to blocks
Use the function: filter_data(data, filters = []). 
Ensure that therere is a filter in the filterdict within the function that corresponds to the blocks and other column specified values that you want to filter by.

You can add filters by adding lines to the dict of this format:
    'filter key name': ['Relevant column name', Relevant row values to filter],

Add the same filters to the dict you define below

* Allways filter by mouse in addition to other filters, if the input data includes multiple animal recordings

In [ ]:
#Filter mice to get unique indexes

# first make the dict structure
mouse_data = {'session 1':{'closedloop': {},'regular': {}, 'no_mismatch':{}}, 'session 2': {'closedloop': {},'regular': {}, 'no_mismatch':{}}}

#Then assign mouse by mouse with filtered data to the dict
for mouse in GRAB_Reg_1.mouseID.unique():
    mouse_data['session 1']['closedloop'][mouse] = filter_data(GRAB_Reg_1, filters = [mouse, 'closed_block'])
    mouse_data['session 1']['regular'][mouse] = filter_data(GRAB_Reg_1, filters = [mouse, 'regular_block'])
    mouse_data['session 1']['no_mismatch'][mouse] = filter_data(GRAB_Reg_1, filters = [mouse, 'normal_block'])
    
for mouse in GRAB_Reg_2.mouseID.unique():
    mouse_data['session 2']['closedloop'][mouse] = filter_data(GRAB_Reg_2, filters = [mouse, 'closed_block'])
    mouse_data['session 2']['regular'][mouse] = filter_data(GRAB_Reg_2, filters = [mouse, 'regular_block'])
    mouse_data['session 2']['no_mismatch'][mouse] = filter_data(GRAB_Reg_2, filters = [mouse, 'normal_block'])

# Align data and save to dict
Run process_mouse_data which will call align_to_event_start and align traces and structure tham into a dictionary.
Specify:
* The trace columns to align and not baseline
* The trace columns to align and basline (can be the same as the former)
* The event column (bool column, halt, No_halt, or other)

In [ ]:
# Example input data (assuming `aligned_data` is structured as needed)
trace_columns_to_baseline = ['470_dfF']  # Columns to baseline
trace_columns_no_baseline = ['movementY', 'movementX']  # Columns to leave unaltered
event_col = 'halt'

# Process the data
aligned_data = process_mouse_data(mouse_data, trace_columns_to_baseline, trace_columns_no_baseline, event_col, [1, 2])

event_col = 'No_halt'
aligned_data_nohalt = process_mouse_data(mouse_data, trace_columns_to_baseline, trace_columns_no_baseline, event_col, [1, 2])

In [ ]:
aligned_data['session 1']['closedloop'][names[0]]

In [ ]:
plot_compare_blocks(aligned_data['session 1'], '470_dfF_bsl')
plot_compare_blocks(aligned_data['session 2'], '470_dfF_bsl')


In [ ]:
plot_compare_blocks(aligned_data['session 1'], 'movementX')
plot_compare_blocks(aligned_data['session 2'], 'movementX')

# Plot across mouse means
Make dicts with mouse mean data depending on session and blocks, and make also a dict with the controls.

Use these to plot the across mouse means using the plot_mean_across_blocks function.

In [ ]:
mouse_mean_session_data = mouse_mean_bsl_data(aligned_data, '470_dfF_bsl')
mouse_mean_control_data = mouse_mean_bsl_data(aligned_data_nohalt, '470_dfF_bsl')



session1_blocks = {'closedloop': mouse_mean_session_data['session 1']['closedloop'],
                  'regular': mouse_mean_session_data['session 1']['regular']}
control1_blocks = {'closedloop': mouse_mean_control_data['session 1']['closedloop']}

session2_blocks = {'closedloop': mouse_mean_session_data['session 2']['closedloop'],
                  'regular': mouse_mean_ession_data['session 2']['regular']}
control2_blocks = {'closedloop': mouse_mean_control_data['session 2']['closedloop']}


In [ ]:
# Defining plotting dicts for relevant blocks
session1_blocks = {'closedloop': mouse_mean_session_data['session 1']['closedloop'],
                  'regular': mouse_mean_session_data['session 1']['regular_block']}
control1_blocks = {'closedloop': mouse_mean_control_data['session 1']['closedloop'],
                  'no_mismatch': mouse_mean_session_data['session 1']['normal_block']}

session2_blocks = {'closedloop': mouse_mean_session_data['session 2']['closedloop'],
                  'regular': mouse_mean_session_data['session 2']['regular_block']}

control2_blocks = {'closedloop': mouse_mean_control_data['session 2']['closedloop'], 
                  'no_mismatch': mouse_mean_session_data['session 2']['normal_block']}


In [ ]:

# Plot the data
plot_mean_across_blocks(session1_blocks, control1_blocks, title="Session 1 blocks")
plot_mean_across_blocks(session2_blocks, control1_blocks, title="Session 2 blocks")


# Compute statistics
Use the compute_trace_statistics to get some paramters from the data. 
It will return the peak, mean, median and standard deviation for each aligned trace chunk during the time windows 0-1s, 1-2s, and -1-0s.
* Specify the aligned trace that you want statistics for
* In addition you can specify some other traces for which you will get the mean during, after, and before (same time intervals)

* Consider doing this for multiple main traces. Then add a column named for example 'trace' where the main trace name is the value of each row. Combe them all in one df.

In [ ]:
# Specify the main trace and co-traces
main_trace = "470_dfF_bsl"
co_traces = ["movementX", "movementY"]
ranges = [(-1, 0), (0, 1), (1, 2)]

# Compute statistics
stats_470_main_df = compute_trace_statistics(aligned_data,trace=main_trace,co_traces=co_traces,ranges=ranges,)


stats_470_nohalt_df = compute_trace_statistics(aligned_data_nohalt,trace=main_trace,co_traces=co_traces,ranges=ranges,)
# Before combinind datasets, introduce a column to seperate the types of data. 
# Can also be control True and False or other specific event that makes the datsets different
stats_470_main_df['halt'] = True
stats_470_nohalt_df['halt'] = False

# Combine both datasets 
all_470_stats_df = pd.concat([stats_470_main_df, stats_470_nohalt_df], ignore_index=True)
all_470_stats_df

In [ ]:
all_470_stats_df.to_csv('saved_data/GRAB_RegMM_all_470_stats_df.csv', index=False)

In [ ]:
##Consider adding to the fitted model a continous variable which is the time column
#EventTime should currently be seconds from session start

##  Possible: Look for correlation between running and fluorescence changes

In [ ]:
subset = filter_data(GRAB_Reg_1, [names[0], 'day1'])

In [ ]:
subset

In [ ]:
# Compute Pearson correlation
pearson_corr, pearson_pval = pearsonr(subset['movementX'], subset['470_dfF'])

# Compute Spearman correlation (handles nonlinear relationships better)
spearman_corr, spearman_pval = spearmanr(subset['movementX'], subset['470_dfF'])

print(f"Pearson correlation: {pearson_corr}, p-value: {pearson_pval}")
print(f"Spearman correlation: {spearman_corr}, p-value: {spearman_pval}")

In [ ]:
resampled = subset.sample(10000, random_state=9)  # Sample 10,000 rows
pearson_corr, pearson_pval = pearsonr(resampled['movementX'], resampled['470_dfF'])
spearman_corr, spearman_pval = spearmanr(resampled['movementX'], resampled['470_dfF'])
print(f"Subset Pearson correlation: {pearson_corr}, p-value: {pearson_pval}")
print(f"Subset Spearman correlation: {spearman_corr}, p-value: {spearman_pval}")